<a href="https://colab.research.google.com/github/nhatpham2016/stocknews/blob/main/evaluation_algorithmic_feedback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation


In [2]:
!pip install langsmith -q
%pip install --upgrade openai -q
%pip install langchain -q
%pip install -U langchain-community -q

from langchain.chat_models import AzureChatOpenAI
import os
from google.colab import userdata

# Setting environment variables from Colab's user data
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["deployment_name"] = userdata.get('deployment_name')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## langsmith stuffs
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('lang_smith_api_key')
os.environ["LANGCHAIN_HUB_API_URL"] = "https://api.hub.langchain.com"
os.environ["LANGCHAIN_HUB_API_KEY"] = userdata.get('lang_smith_api_key')
project_name = "dataset_sales_strategies_test_v1"

In [3]:
%pip install -qU langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.8 MB/s eta 0:00:00


We will create some example runs in this project to get us off on a shared footing.

In [4]:
from langsmith import Client
from datetime import datetime

client = Client()


#Evaluation Criteria prompt setup

In [75]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, PromptTemplate, HumanMessagePromptTemplate


prompt = ChatPromptTemplate(
    input_variables=['prediction', 'question'],
    input_types={},
    partial_variables={},
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template="You are grading customer inquiries related to our marketing and sales strategy. The focus is on evaluating how well the responses align with business objectives. You must rate the responses using the following criteria: market relevance, strategic exploration depth, competitive positioning clarity, and actionable insights."
            ),
            additional_kwargs={}
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['prediction', 'question'],
                input_types={},
                partial_variables={},
                template='Grade the following question on a scale from 0 to 5, the higher the score the better the response:\n\n<question>\n{question}\n</question>\n\nThe bot responded:\n\n<response>\n{prediction}\n</response>\n\nPlease evaluate based on the following criteria:\n1. Market Condition Relevance\n2. Strategic Exploration Depth\n3. Comparative Strategy Clarity\n4. Insight Specificity'
            ),
            additional_kwargs={}
        ),
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                input_types={},
                partial_variables={},
                template='Respond in JSON format, giving each value a score between 0 and 5 for the following keys: "market_condition_relevance", "strategic_exploration_depth", "comparative_strategy_clarity", "insight_specificity".'
            ),
            additional_kwargs={}
        )
    ]
)


In [76]:


llm = AzureChatOpenAI(
        deployment_name= os.environ["deployment_name"],
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_version=os.getenv("OPENAI_API_VERSION")
    )


In [77]:
from langchain_openai import ChatOpenAI

In [78]:

evaluator =  ChatOpenAI(
    model="gpt-4o-mini")

In [79]:
from langchain_core.tracers.context import collect_runs

from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser


# Define the chain with updated scoring criteria
chain = (
    prompt
    | evaluator.bind(
        functions=[
            {
                "name": "submit_scores",
                "description": "Submit the graded scores for a user question and bot response. Higher the score the better",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "market_condition_relevance": {
                            "type": "integer",
                            "minimum": 0,
                            "maximum": 5,
                            "description": "Score indicating how well the answer addresses the Market and Company Condition Analysis",
                        },
                        "strategic_exploration_depth": {
                            "type": "integer",
                            "minimum": 0,
                            "maximum": 5,
                            "description": "Score reflecting the depth of Guided Strategic Exploration in the answer.",
                        },
                        "comparative_strategy_clarity": {
                            "type": "integer",
                            "minimum": 0,
                            "maximum": 5,
                            "description": "Score indicating the clarity and completeness of Comparative Strategy Evaluation in the response.",
                        },
                        "insight_specificity": {
                            "type": "integer",
                            "minimum": 0,
                            "maximum": 5,
                            "description": "Score indicating the level of specificity and precision in providing Data-Driven Insights.",
                        },
                    },
                    "required": [
                        "market_condition_relevance",
                        "strategic_exploration_depth",
                        "comparative_strategy_clarity",
                        "insight_specificity",
                    ],
                },
            }
        ]
    ) | JsonOutputFunctionsParser()
)



In [80]:
zero_shot_input = ''' Hey, can you pull together the SWOT analysis for our sales department? I want to present it at the next management meeting.
'''

zero_shot_output = '''Sure! Do you have any specific strengths in mind that we should highlight?'''

In [81]:
result = chain.invoke(
                {
                    "question": zero_shot_input ,
                    "prediction": zero_shot_output
                }
            )



In [82]:
# Record feedback for each key in the result
# we don't use zero it is too simple
for feedback_key, value in result.items():
    score = int(value) / 5  # Normalize the score to 0-1 scale
    print(f"Recording feedback for key: {feedback_key}, Score: {score}")

Recording feedback for key: market_condition_relevance, Score: 0.4
Recording feedback for key: strategic_exploration_depth, Score: 0.2
Recording feedback for key: comparative_strategy_clarity, Score: 0.2
Recording feedback for key: insight_specificity, Score: 0.4


# Prompt Techinique tests

In [25]:
import pandas as pd

# Example conversation data
conversation = [
    {"human": "we are IT sales", "ai": "hello there"},
    {"human": "we target small business", "ai": "Great, targeting small businesses is a valuable piece of information. Moving on to the competitive landscape: Who are your main competitors in the market?"},
    {"human": "company XYZ", "ai": None},  # No response here, skipping
    {"human": "we have bad sales last week", "ai": "Thank you for sharing that insight. To delve deeper into internal factors: What specific challenges or obstacles are impacting your business operations currently?"},
    {"human": "we don't have enough salespersons", "ai": "Understood. Improving sales efficiency is crucial. Now, focusing on external factors: What market trends or opportunities do you see in your industry that could impact your business positively?"},
    {"human": "our economy is getting bad", "ai": "Thank you for highlighting that concern about the economy. Moving on to competitors and risks: What specific risks or challenges do your competitors pose to your business in the current market environment?"},
    {"human": "they screw up our sales", "ai": '''Thank you for sharing that insight about the impact of competitors on your sales. Let's summarize the information we have gathered so far to create a comprehensive SWOT analysis:

**SWOT Analysis Summary:**
- **Strengths:**
  - IT sales expertise
  - Targeting small businesses

- **Weaknesses:**
  - Inadequate sales personnel
  - Recent poor sales performance

- **Opportunities:**
  - Market trends or opportunities in the industry
  - *Please provide any additional opportunities you have identified*

- **Threats:**
  - Economic downturn
  - Competitors impacting sales

How does this summary align with your business's current situation and goals? Feel free to provide any additional information or insights to enhance the analysis. '''},  # No response here, skipping
]

# Manually entered strategies
strategies = ["Introduction", "Targeting", "Sales Report", "Challenges", "Staff Shortage", "Market Trends"]

def evaluate_pair(human_input, ai_output):
    # Simulate the result of the evaluation chain (replace with actual logic)
    result = chain.invoke(
        {
            "question": human_input,
            "prediction": ai_output
        }
    )

    # Create a dictionary to store feedback for each key
    feedback_scores = {}

    # Record feedback for each key in the result
    for feedback_key, value in result.items():
        score = int(value) / 5  # Normalize the score to 0-1 scale
        feedback_scores[feedback_key] = score

    return feedback_scores


# DataFrame columns
columns = ["Strategy", "Human", "AI", "market_condition_relevance", "strategic_exploration_depth", "comparative_strategy_clarity", "insight_specificity"]

# Collect data for DataFrame
data = []

# Iterate through conversation pairs
for i, pair in enumerate(conversation):
    human_input = pair.get("human")
    ai_output = pair.get("ai")

    # Only evaluate pairs where AI provided a response
    if ai_output is not None:
        result = evaluate_pair(human_input, ai_output)

        # Collect row data: Strategy, Human, AI, Scores
        row = ['Chain-of-Thought Prompting (CoT)', human_input, ai_output] + [result.get(key) for key in result]
        data.append(row)

# Create DataFrame
df_cot_simple = pd.DataFrame(data, columns=columns)

# Display the DataFrame


In [26]:
def process_conversation(conversation, strategy_name):
    # DataFrame columns
    columns = ["Strategy", "Human", "AI", "market_condition_relevance", "strategic_exploration_depth", "comparative_strategy_clarity", "insight_specificity"]

    # Collect data for DataFrame
    data = []

    # Iterate through conversation pairs
    for i, pair in enumerate(conversation):
        human_input = pair.get("human")
        ai_output = pair.get("ai")

        # Only evaluate pairs where AI provided a response
        if ai_output is not None:
            result = evaluate_pair(human_input, ai_output)

            # Collect row data: Strategy, Human, AI, Scores
            row = [strategy_name, human_input, ai_output] + [result.get(key) for key in result]
            data.append(row)

    # Create DataFrame
    df = pd.DataFrame(data, columns=columns)

    return df

#

In [27]:
df_cot_simple = process_conversation(conversation, "Chain-of-Thought Prompting (CoT)")

In [28]:
df_cot_simple

,Strategy,Human,AI,market_condition_relevance,strategic_exploration_depth,comparative_strategy_clarity,insight_specificity
0,Chain-of-Thought Prompting (CoT),we are IT sales,hello there,0.0,0.2,0.2,0.2
1,Chain-of-Thought Prompting (CoT),we target small business,"Great, targeting small businesses is a valuabl...",0.8,0.6,0.4,0.2
2,Chain-of-Thought Prompting (CoT),we have bad sales last week,Thank you for sharing that insight. To delve d...,0.4,0.6,0.4,0.6
3,Chain-of-Thought Prompting (CoT),we don't have enough salespersons,Understood. Improving sales efficiency is cruc...,0.8,0.6,0.0,0.0
4,Chain-of-Thought Prompting (CoT),our economy is getting bad,Thank you for highlighting that concern about ...,0.4,0.6,0.8,0.6
5,Chain-of-Thought Prompting (CoT),they screw up our sales,Thank you for sharing that insight about the i...,0.8,0.6,0.6,0.6


## React

In [30]:
conversation = [
    {"human": "hi", "ai": "What does your business do? Please describe your products or services."},
    {"human": "We do IT sales.", "ai": "What is the main goal or objective for your business in the near future? (e.g., growth, expansion, solving a challenge)"},
    {"human": "We want to grow.", "ai": "Who are your primary customers or target audience?"},
    {"human": "We aim to target medium to mid-size companies.", "ai": "What are the key areas in which your team excels?"},
    {"human": "Various ones, but we are good at selling.", "ai": "What resources (people, technology, assets) are essential to running your business?"},
    {"human": "We have the best people in town.", "ai": "What processes or operations within your business are running efficiently?"},
    {"human": "We are fast at delivering products and services at reasonable prices.", "ai": "What are some of your recent achievements or milestones?"},
    {"human": "None.", "ai": "What challenges have you faced recently within your operations?"},
    {"human": "We don’t have enough salespeople.", "ai": "What trends are you noticing in your industry or market?"},
    {"human": "Pretty good, very high growth.", "ai": "What are some risks or threats in your industry that could affect your business?"},
    {"human": "Intensive competition.", "ai": "What are some new opportunities in the market that you are looking to explore?"},
    {"human": "More tailoring to brick-and-mortar businesses.", "ai": ''' Based on the information gathered, here is a summary of your SWOT analysis:

- **Strengths**: Strong sales capabilities, fast delivery of products and services at reasonable prices, and a talented team.
- **Weaknesses**: Insufficient sales personnel and lack of recent achievements or milestones.
- **Opportunities**: High growth trends in the market and potential to tailor services to brick-and-mortar businesses.
- **Threats**: Intense competition in the IT sales industry.'''},
]


In [31]:
df_react = process_conversation(conversation, "React Prompting")

In [32]:
df_react

,Strategy,Human,AI,market_condition_relevance,strategic_exploration_depth,comparative_strategy_clarity,insight_specificity
0,React Prompting,hi,What does your business do? Please describe yo...,0.4,0.2,0.4,0.2
1,React Prompting,We do IT sales.,What is the main goal or objective for your bu...,0.8,0.6,0.4,0.2
2,React Prompting,We want to grow.,Who are your primary customers or target audie...,0.8,0.4,0.2,0.4
3,React Prompting,We aim to target medium to mid-size companies.,What are the key areas in which your team excels?,0.8,0.4,0.2,0.2
4,React Prompting,"Various ones, but we are good at selling.","What resources (people, technology, assets) ar...",0.2,0.4,0.0,0.4
5,React Prompting,We have the best people in town.,What processes or operations within your busin...,0.2,0.4,0.4,0.6
6,React Prompting,We are fast at delivering products and service...,What are some of your recent achievements or m...,0.4,0.2,0.4,0.2
7,React Prompting,None.,What challenges have you faced recently within...,0.0,0.2,0.0,0.2
8,React Prompting,We don’t have enough salespeople.,What trends are you noticing in your industry ...,0.8,0.6,0.4,0.2
9,React Prompting,"Pretty good, very high growth.",What are some risks or threats in your industr...,0.6,0.4,0.8,0.6


## ReAct Prompting with More Reflections

In [33]:
conversation = [
    {"human": "hello there", "ai": "What type of business are you involved in, and what products or services do you offer?"},
    {"human": "we are IT sales :))", "ai": "What specific IT products or services do you specialize in, and who is your target market? Additionally, how long have you been in this industry?"},
    {"human": "we sell AI prompts and we target small to medium-sized companies. We have been in the industry for 5 years.", "ai": "What are some of the key internal processes or resources that you believe support your sales efforts? For example, consider your team skills, technology, or customer relationships."},
    {"human": "Our engineering team is pretty good. We also manage customer relationships well and use cutting-edge technologies.", "ai": "What challenges have you faced in your sales efforts, whether related to competition, market demand, or internal processes?"},
    {"human": "We have intensive competition. The market demand is growing. For internal processes, we are small, which makes us fast but we may lack some standards.", "ai": "What are the key trends in the market that you believe could present new opportunities for your business? Additionally, what external factors do you think could pose significant threats to your success in the IT sales sector?"},
    {"human": "Exploding demand from everyone. But also the competition, and the technological difficulties like hallucinations in large language models (LLMs).", "ai": "What is your current financial performance, including revenue, profit margins, and growth rates? Additionally, how do you anticipate these metrics changing in the near future?"},
    {"human": "We are in good shape, but I expect them to grow.", "ai": "What specific goals or strategic focuses do you have for the next few years, particularly in relation to expanding your customer base or enhancing your offerings?"},
    {"human": "We are trying to obtain more than 200 users.", "ai": '''Based on the information gathered, here is a summary of your SWOT analysis:

- **Strengths**:
  - Strong engineering team
  - Effective customer relationship management
  - Utilization of cutting-edge technologies

- **Weaknesses**:
  - Lack of standardized internal processes due to being a small team
  - Potential challenges in scaling operations

- **Opportunities**:
  - Increasing demand for AI prompts among small to medium-sized companies
  - Potential to expand customer base significantly (targeting 200 more users)

- **Threats**:
  - Intense competition in the IT sales sector
  - Challenges related to technology, such as hallucinations in language models

If you have any specific areas you'd like to explore further or any additional insights you'd like to share, please let me know! '''}  # No AI response here, skipping
]


In [34]:
df_react_with_reflections = process_conversation(conversation, "React Prompting With Reflections")

In [35]:
df_react_with_reflections

,Strategy,Human,AI,market_condition_relevance,strategic_exploration_depth,comparative_strategy_clarity,insight_specificity
0,React Prompting With Reflections,hello there,"What type of business are you involved in, and...",0.4,0.4,0.2,0.2
1,React Prompting With Reflections,we are IT sales :)),What specific IT products or services do you s...,0.8,0.6,0.4,0.0
2,React Prompting With Reflections,we sell AI prompts and we target small to medi...,What are some of the key internal processes or...,0.6,0.8,0.4,0.6
3,React Prompting With Reflections,Our engineering team is pretty good. We also m...,What challenges have you faced in your sales e...,0.4,0.6,0.2,0.0
4,React Prompting With Reflections,We have intensive competition. The market dema...,What are the key trends in the market that you...,1.0,0.8,0.6,0.4
5,React Prompting With Reflections,Exploding demand from everyone. But also the c...,"What is your current financial performance, in...",0.8,0.4,0.2,0.2
6,React Prompting With Reflections,"We are in good shape, but I expect them to grow.",What specific goals or strategic focuses do yo...,0.4,0.6,0.4,0.2
7,React Prompting With Reflections,We are trying to obtain more than 200 users.,"Based on the information gathered, here is a s...",0.8,0.6,0.6,0.4


## function prompts

In [36]:
conversation = [
    {"human": "hello", "ai": "Hello! How can I assist you today? Are you looking to analyze a business, or do you have a specific project in mind?"},
    {"human": "yes I do", "ai": "Great! Could you please provide a brief overview of the business or project you're looking to analyze? This will help me understand the context better."},
    {"human": "I am in IT sales", "ai": "That’s helpful to know! In the IT sales sector, there can be many factors at play. Could you tell me more about your specific offerings or products? Additionally, who are your primary customers or target markets? This information will help me understand your business better."},
    {"human": "we target medium and small customers. We sell AI prompts", "ai": "Thank you for sharing that! Targeting medium and small customers with AI prompts is a great niche.\nTo further refine our analysis, could you provide more details on the following:\n\nCompetitive Landscape: Who are your main competitors in this space? What sets you apart from them?\nSales Process: What does your sales process look like? Are there specific channels you use to reach your customers?\nChallenges: What challenges or obstacles are you currently facing in your sales efforts?\nMarket Trends: Are there any trends or shifts in the market that you've noticed recently?\nCustomer Feedback: What feedback do you receive from your customers about your products?\nThis information will help us create a more comprehensive analysis."},
    {"human": "we have very intensive competition\nour sales process is doing okay, just doing via sale channels\nwe don't have enough sales staff\nhigh inflation and such\nsome say we are good, some say we are bad", "ai": "Thank you for providing that detailed information! Let's break it down a bit more:\n\nCompetitive Landscape: You mentioned that there is very intensive competition. It's important to know who your main competitors are and what differentiates you from them.\nSales Process: You stated that the sales process is doing okay and is conducted via sales channels. Are there specific channels that work better for you?\nChallenges: You highlighted a lack of sufficient sales staff as a challenge. Are there other challenges you face, such as product awareness or customer acquisition?\nMarket Trends: You noted high inflation as a market trend. Are there any other trends or shifts in technology or customer behavior you've observed?\nCustomer Feedback: It seems you're receiving mixed feedback from customers. Can you elaborate on what specific aspects they praise or criticize?\nOnce I have a clearer picture of these elements, I can help you perform a SWOT analysis."},
    {"human": "we have better prompts and simpler but higher quality\nemail is the best\nwe do have some, like new products launching so rapidly\nthat's all for now, maybe higher competition\nthey say we are good at prompting lol", "ai": '''Thank you for the detailed responses! Let's summarize the information you've provided:

1. **Competitive Landscape**: You have better prompts that are simpler and of higher quality compared to your competitors.
2. **Sales Process**: Email is your most effective sales channel.
3. **Challenges**: You face challenges like a lack of sales staff and rapid product launches.
4. **Market Trends**: High inflation and increased competition are notable trends.
5. **Customer Feedback**: Customers generally find your prompting skills good, but feedback is mixed.

Now, I have enough information to create a SWOT analysis for your IT sales business focusing on AI prompts. Here’s the summary:

### SWOT Analysis

- **Strengths**:
  - High-quality and simpler AI prompts that set you apart from competitors.
  - Effective sales channel through email, which can facilitate direct communication with customers.

- **Weaknesses**:
  - Insufficient sales staff, limiting your ability to reach more customers and expand sales efforts.
  - Challenges associated with rapid product launches, which may lead to operational inefficiencies.

- **Opportunities**:
  - Potential to capitalize on the growing interest in AI solutions among medium and small businesses.
  - Explore additional marketing strategies or sales channels to enhance customer engagement and sales reach.

- **Threats**:
  - High inflation impacting customer budgets and spending.
  - Intense competition which may affect market share and pricing strategies.

If there are any specific areas you want to dive deeper into or if you have additional insights to add, feel free to share! '''}  # No AI response here, skipping
]


In [37]:
df_function_prompts = process_conversation(conversation, "Function Prompting")

In [38]:
df_function_prompts

,Strategy,Human,AI,market_condition_relevance,strategic_exploration_depth,comparative_strategy_clarity,insight_specificity
0,Function Prompting,hello,Hello! How can I assist you today? Are you loo...,0.6,0.4,0.4,0.2
1,Function Prompting,yes I do,Great! Could you please provide a brief overvi...,0.8,0.6,0.4,0.2
2,Function Prompting,I am in IT sales,That’s helpful to know! In the IT sales sector...,0.8,0.6,0.4,0.2
3,Function Prompting,we target medium and small customers. We sell ...,Thank you for sharing that! Targeting medium a...,0.8,1.0,1.0,0.8
4,Function Prompting,we have very intensive competition\nour sales ...,Thank you for providing that detailed informat...,0.8,0.6,0.6,0.8
5,Function Prompting,we have better prompts and simpler but higher ...,Thank you for the detailed responses! Let's su...,0.8,0.8,1.0,0.8


## Diagrams of thoughts

In [52]:
data = [{'role': 'user', 'content': 'hi there'}, {'role': 'assistant', 'content': "Hello! I'm glad you're here. To get started on understanding your business better, could you tell me about your business’s core offerings and how you differentiate from your competitors?"}, {'role': 'user', 'content': 'we are IT sales and we have great in our DNA'}, {'role': 'assistant', 'content': 'That\'s a great start! Your focus on IT sales and your emphasis on having "great in your DNA" suggests a strong foundation. \n\nTo dive deeper, can you share:\n1. "What specific products or services do you offer within IT sales?"\n2. "Who are your key customer segments, and what needs do you fulfill for them?"\n3. "How has your business performed over the past year in terms of growth, revenue, and market presence?"'}, {'role': 'user', 'content': '1) we sell prompts, they are very very good \n2) we target small and medium size company, we help people become more productive\n3) we growth better than last year, yoy about 20%'}, {'role': 'assistant', 'content': 'Thank you for sharing that information! It sounds like you have a solid product offering with a focus on enhancing productivity for small and medium-sized businesses, and a healthy year-over-year growth rate of 20%.\n\nTo further refine our understanding, let\'s explore some internal factors. \n\n1. "What unique resources or capabilities give your business a competitive edge?"\n2. "How does your team’s expertise contribute to your success in the market?"\n3. "Are there any processes or technologies that help your business operate more efficiently than others?"'}, {'role': 'user', 'content': 'Unique Resources: Our AI’s real-time data analysis gives us a distinct advantage in optimizing sales processes.\nTeam Expertise: Our team’s deep knowledge in AI and sales automation drives innovation and keeps us competitive.\nEfficient Operations: Automated workflows and machine learning models streamline our operations, improving efficiency and reducing costs.\n\n\n\n\n\n'}, {'role': 'assistant', 'content': 'Thank you for providing those insights! It\'s clear that your use of AI for real-time data analysis, along with your team\'s expertise in AI and sales automation, positions you well in the market. Additionally, your automated workflows enhance efficiency and cost-effectiveness.\n\nNow, let\'s delve into some potential challenges your business might face. \n\n1. "What internal challenges do you face that slow down your business growth?"\n2. "Are there any skills or resources you feel your business lacks to compete effectively?"\n3. "How do you manage inefficiencies in your current operations?"'}, {'role': 'user', 'content': 'Challenges to Growth: Scaling our infrastructure quickly enough to meet demand has been a key challenge.\nSkills or Resources Lacking: We need more specialized talent in data science and AI to remain competitive.\nManaging Inefficiencies: We’re constantly refining our processes and investing in automation to minimize bottlenecks.\n\n\n\n\n\n'}, {'role': 'assistant', 'content': 'Thank you for your detailed responses. It’s evident that you’re aware of both your strengths and the challenges you face. Your focus on AI and automation is a strong point, but scaling infrastructure and talent acquisition are critical areas to address.\n\nNow, let’s shift our focus to external factors that could impact your business. \n\n1. "What market trends do you see that might offer growth opportunities for your business?"\n2. "Are there any new customer segments or geographic markets you are considering expanding into?"\n3. "How could partnerships or collaborations open new doors for your business?"'}, {'role': 'user', 'content': 'Market Trends: The increasing demand for AI-driven sales solutions and automation presents significant growth opportunities.\nNew Markets: We are exploring expansion into the healthcare and fintech sectors, as well as the APAC region.\nPartnerships: Collaborating with industry leaders in CRM and sales tools could enhance our offerings and access new customer bases.\n\n\n\n\n\n'}, {'role': 'assistant', 'content': 'Thank you for sharing those insights. It\'s clear that there are promising trends in AI-driven sales solutions, and your exploration of new markets and potential partnerships is strategic.\n\nNow, let’s examine the external challenges your business might face. \n\n1. "What external factors (such as competitors or market changes) do you perceive as potential risks to your business?"  \n2. "Are there regulatory or economic changes that could affect your business negatively?"  \n3. "How do you see your competitors reacting to changes in the market, especially with the rise of AI and automation?"'}, {'role': 'user', 'content': 'External Risks: Increased competition in AI-driven sales platforms and market saturation could limit our growth.\nRegulatory Changes: Stricter data privacy regulations could impact our ability to collect and analyze customer data effectively.\nCompetitor Reactions: Competitors are rapidly adopting AI and automation, pushing us to innovate faster to maintain our market position.\n\n\n\n\n\n'}, {'role': 'assistant', 'content': 'Thank you for providing such comprehensive information about your business. The insights you’ve shared will be instrumental in crafting a thorough analysis. \n\nLet\'s now focus on understanding your customers and market perspective to further enrich our analysis.\n\n1. "How do your customers perceive your brand compared to your competitors?"\n2. "What are the primary reasons your customers choose your products or services over others?"\n3. "Have you noticed any shifts in customer preferences or buying behaviors recently?"'}, {'role': 'user', 'content': 'Brand Perception: Customers see us as an innovative and reliable solution compared to more traditional competitors.\nWhy They Choose Us: Our AI-powered insights and automation tools provide greater efficiency and accuracy in sales processes.\nShifts in Behavior: Customers are increasingly seeking more personalized, automated sales solutions to streamline their operations.\n\n\n\n\n\n'}, {'role': 'assistant', 'content': 'Thank you for sharing such valuable insights regarding your customers and market perspective. It’s great to see that your brand is recognized for innovation and reliability, especially as customers increasingly seek personalized solutions.\n\nNow, let’s take a look at your financial overview to understand how financial performance and resource allocation play into your business strategy:\n\n1. "How has your business performed financially over the past year, and what are your projections moving forward?"\n2. "Are there any financial constraints or challenges that are limiting your growth?"\n3. "What key investments have been most effective in driving growth?"'}, {'role': 'user', 'content': 'Past Year Performance: We’ve experienced strong growth, with a 25% increase in revenue over the past year.\nFinancial Constraints: Scaling costs for infrastructure and hiring have slowed down expansion efforts.\nKey Investments: Investments in AI development and sales automation tools have been the most effective in driving growth.\n\n\n\n\n\n'}, {'role': 'assistant', 'content': 'Thank you for the detailed financial overview. It\'s encouraging to see strong revenue growth, although I note the challenges related to scaling costs and hiring.\n\nNext, let’s explore your strategic focus and goals to understand where you’re headed as a business:\n\n1. "What are your business’s short-term and long-term strategic goals?"\n2. "Which growth strategies are you currently pursuing, and why?"\n3. "Are there any key initiatives you are focusing on to drive growth?"'}, {'role': 'user', 'content': 'Short-Term & Long-Term Goals: Short-term, we aim to enhance AI features; long-term, we plan to expand globally.\nGrowth Strategies: We’re focusing on partnerships and product innovation to capture new markets and improve customer retention.\nKey Initiatives: Expanding our AI capabilities and entering new sectors are our primary initiatives for driving growth.\n\n\n\n\n\n'}, {'role': 'assistant', 'content': 'Thank you for providing a wealth of information about your business. It paints a clear picture of your current position and strategic direction. \n\nNow, let’s take a look at the role of external relationships in your business strategy to finalize our understanding:\n\n1. "What role do external partnerships or supplier relationships play in your business strategy?"\n2. "Are there any key alliances that provide you with a competitive advantage?"\n3. "How have external relationships helped or hindered your business growth?"\n\nFeel free to elaborate on these points!'}, {'role': 'user', 'content': 'Role in Strategy: External partnerships are central to expanding our product offerings and reaching new markets.\nKey Alliances: Strategic alliances with CRM and automation platforms give us a competitive advantage by enhancing integrations.\nImpact on Growth: These relationships have accelerated growth by expanding our customer base, though dependence on suppliers can sometimes delay product updates.\n\n\n\n\n\n'}, {'role': 'assistant', 'content': "Thank you for the comprehensive insights you've shared about your business. Here’s a critique of the information gathered so far:\n\n1. **Market Trends and Growth Opportunities**: You mentioned the increasing demand for AI-driven solutions and the exploration of new markets like healthcare and fintech. However, it would be beneficial to explore specific customer needs within these sectors to understand the nuances of these opportunities.\n\n2. **External Risks**: You identified increased competition and regulatory changes as significant risks. It might also be valuable to assess potential economic downturns or shifts in technology that could impact your operations.\n\n3. **Customer Preferences**: Your insights into customer preferences are strong, particularly regarding the desire for personalized solutions. However, further exploration into customer feedback mechanisms and how you adapt based on that feedback could provide deeper insights.\n\n4. **Financial Overview**: You shared impressive growth metrics and challenges related to scaling costs. It would be useful to explore your cost structure in more detail to identify specific areas for potential efficiency improvements.\n\n5. **Strategic Goals**: Your short-term and long-term goals are clear. However, considering how these goals align with market trends, customer preferences, and potential risks could provide a more holistic view of your strategic positioning.\n\n6. **External Relationships**: You detailed the importance of partnerships, which is great. It might be beneficial to evaluate how these relationships have evolved over time and any lessons learned that could inform future collaborations.\n\nWith these points in mind, I suggest a few additional questions to further clarify and deepen the analysis:\n\n- For market trends, can you specify how customer needs differ in the healthcare and fintech sectors?\n- Are there economic factors you believe could impact the growth of AI-driven sales solutions in your industry?\n- How do you gather and incorporate customer feedback into your product development process?\n- What specific areas of your cost structure are you looking to optimize?\n- In terms of partnerships, can you share any specific success stories or challenges you’ve faced in these collaborations?\n\nThis additional information will help ensure that we have a robust understanding of your business as we move to the SWOT analysis stage."}, {'role': 'user', 'content': '### Sector-Specific Customer Needs:\n\n1. **Healthcare Needs:** In healthcare, customers require AI solutions that prioritize compliance, security, and data privacy while streamlining patient engagement and administrative tasks.\n2. **Fintech Needs:** Fintech clients focus on AI solutions that enhance transaction speed, fraud detection, and personalized financial recommendations.\n\n### Economic Factors:\n\n3. **Economic Impact:** Rising interest rates and inflation could tighten budgets, leading businesses to be more selective in adopting AI-driven sales solutions, while others may accelerate adoption to cut operational costs.\n\n### Customer Feedback:\n\n4. **Incorporating Feedback:** We use surveys, direct feedback channels, and customer advisory boards to gather insights, which we incorporate into product development sprints for iterative improvements.\n\n### Cost Structure Optimization:\n\n5. **Areas for Optimization:** We are focusing on optimizing infrastructure costs, cloud services, and operational workflows to improve scalability and reduce overhead.\n\n### Partnership Insights:\n\n6. **Success Stories & Challenges:** A recent success involved partnering with a CRM provider, which led to a 15% increase in new clients. However, navigating differing timelines and integration complexities has been a key challenge in some collaborations.'},
 {'role': 'assistant', 'content': '''
 **SWOT Analysis**

Based on the comprehensive information gathered, here’s a structured SWOT analysis for your IT sales business:

### **Strengths:**
- **Innovative Solutions**: Your AI-driven real-time data analysis and automation tools give you a competitive edge in optimizing sales processes.
- **Expert Team**: A knowledgeable team in AI and sales automation drives innovation and keeps you competitive.
- **Efficient Operations**: Automated workflows and machine learning models enhance operational efficiency, reducing costs and bottlenecks.

### **Weaknesses:**
- **Scaling Challenges**: Difficulty in quickly scaling infrastructure to meet demand can hinder growth.
- **Talent Gaps**: A need for more specialized talent in data science and AI may limit your ability to innovate and compete effectively.
- **Dependency on Suppliers**: Relying on external partners may lead to delays in product updates and could impact responsiveness to market changes.

### **Opportunities:**
- **Market Trends**: Increasing demand for AI-driven sales solutions presents significant growth opportunities, especially in sectors like healthcare and fintech.
- **Expansion Potential**: Exploring new customer segments and geographic markets (e.g., APAC) offers avenues for growth.
- **Partnerships**: Collaborating with industry leaders in CRM and sales tools could enhance offerings and access new customer bases.

### **Threats:**
- **Intense Competition**: The rise of AI-driven sales platforms and market saturation could limit growth and pressure margins.
- **Regulatory Challenges**: Stricter data privacy regulations may affect the ability to collect and analyze customer data effectively.
- **Economic Factors**: Rising interest rates and inflation could tighten budgets, making businesses more selective about adopting new technologies.

---

This SWOT analysis presents a balanced view of your business's current landscape, highlighting where you can leverage strengths and opportunities while remaining vigilant about weaknesses and threats. If there are any areas you’d like to further explore or refine, feel free to let me know!
 '''}

        ]

In [53]:
def map_roles(conversation):
    mapped_conversation = []
    for i in range(0, len(conversation), 2):
        # Ensure we have pairs (human, ai)
        if i + 1 < len(conversation):
            human_input = conversation[i]['content'] if conversation[i]['role'] == 'user' else None
            ai_output = conversation[i + 1]['content'] if conversation[i + 1]['role'] == 'assistant' else None
            mapped_conversation.append({'human': human_input, 'ai': ai_output})
    return mapped_conversation


In [54]:
convertdata = map_roles(data)
df_diagrams_of_thoughts  = process_conversation(convertdata, "diagrams_of_thoughts")

In [55]:
df_diagrams_of_thoughts

,Strategy,Human,AI,market_condition_relevance,strategic_exploration_depth,comparative_strategy_clarity,insight_specificity
0,diagrams_of_thoughts,hi there,Hello! I'm glad you're here. To get started on...,0.8,0.6,0.4,0.2
1,diagrams_of_thoughts,we are IT sales and we have great in our DNA,That's a great start! Your focus on IT sales a...,0.8,0.6,0.4,0.4
2,diagrams_of_thoughts,"1) we sell prompts, they are very very good \n...",Thank you for sharing that information! It sou...,0.8,0.6,0.6,0.4
3,diagrams_of_thoughts,Unique Resources: Our AI’s real-time data anal...,Thank you for providing those insights! It's c...,0.8,0.6,0.6,0.4
4,diagrams_of_thoughts,Challenges to Growth: Scaling our infrastructu...,Thank you for your detailed responses. It’s ev...,0.8,0.6,0.6,0.6
5,diagrams_of_thoughts,Market Trends: The increasing demand for AI-dr...,Thank you for sharing those insights. It's cle...,0.8,0.6,0.6,0.4
6,diagrams_of_thoughts,External Risks: Increased competition in AI-dr...,Thank you for providing such comprehensive inf...,1.0,0.8,0.6,0.8
7,diagrams_of_thoughts,Brand Perception: Customers see us as an innov...,Thank you for sharing such valuable insights r...,0.8,0.6,0.4,0.4
8,diagrams_of_thoughts,Past Year Performance: We’ve experienced stron...,Thank you for the detailed financial overview....,0.8,1.0,0.8,0.6
9,diagrams_of_thoughts,"Short-Term & Long-Term Goals: Short-term, we a...",Thank you for providing a wealth of informatio...,0.8,0.6,0.4,0.6


In [68]:
df_all = pd.concat([df_cot_simple, df_react, df_react_with_reflections, df_function_prompts, df_diagrams_of_thoughts])

#Results

In [83]:
## the higher the score the better
df_avg = df_all.groupby('Strategy')[['market_condition_relevance',
                                     'strategic_exploration_depth',
                                     'comparative_strategy_clarity',
                                     'insight_specificity']].mean()

df_avg


,market_condition_relevance,strategic_exploration_depth,comparative_strategy_clarity,insight_specificity
Strategy,,,,
Chain-of-Thought Prompting (CoT),0.533333,0.533333,0.400000,0.366667
Function Prompting,0.766667,0.666667,0.633333,0.500000
React Prompting,0.550000,0.400000,0.333333,0.350000
React Prompting With Reflections,0.650000,0.600000,0.375000,0.250000
diagrams_of_thoughts,0.833333,0.700000,0.600000,0.550000
